In [ ]:
import polars as pl
import geopandas as gpd
from os.path import exists
import rtsvg
rt = rtsvg.RACETrack()
from shapely import Polygon, MultiPolygon
shapefile_path = "../../cb_2018_us_state_20m/cb_2018_us_state_20m/cb_2018_us_state_20m.shp"
gdf = gpd.read_file(shapefile_path)

# Polygon Example
_shape_ = gdf['geometry'][1] # For Polygons
len(_shape_.exterior.coords)
type(_shape_), isinstance(_shape_, Polygon), isinstance(_shape_, MultiPolygon)

# MultiPolygon Example
_shape_ = gdf['geometry'][0]
len(_shape_.boundary.geoms[1].coords)
type(_shape_), isinstance(_shape_, Polygon), isinstance(_shape_, MultiPolygon)

gdf.head()

In [ ]:
_lu_ = {'fm':[], 'to':[]}
pos  = {}
for i in range(len(gdf)):
    _shape_ = gdf['geometry'][i]
    if isinstance(_shape_, Polygon):
        for j in range(len(_shape_.exterior.coords)):
            _xy0_     = _shape_.exterior.coords[j]
            _xy1_     = _shape_.exterior.coords[(j+1)%len(_shape_.exterior.coords)]
            _xy0_str_ = str(_xy0_)
            _xy1_str_ = str(_xy1_)
            pos[_xy0_str_] = _xy0_
            pos[_xy1_str_] = _xy1_
            _lu_['fm'].append(_xy0_str_), _lu_['to'].append(_xy1_str_)
    elif isinstance(_shape_, MultiPolygon):
        for k in range(len(_shape_.boundary.geoms)):
            for j in range(len(_shape_.boundary.geoms[k].coords)):
                _xy0_     = _shape_.boundary.geoms[k].coords[j]
                _xy1_     = _shape_.boundary.geoms[k].coords[(j+1)%len(_shape_.boundary.geoms[k].coords)]
                _xy0_str_ = str(_xy0_)
                _xy1_str_ = str(_xy1_)
                pos[_xy0_str_] = _xy0_
                pos[_xy1_str_] = _xy1_
                _lu_['fm'].append(_xy0_str_), _lu_['to'].append(_xy1_str_)
    else: raise Exception(f'Unknown Type {type(_shape_)}')

df       = pl.DataFrame(_lu_)
_params_ = {'relationships':[('fm','to')], 'pos':pos}

_filename_ = '../../state_layout.parquet'

_igp_ = rt.interactiveGraphPanel(df, ln_params=_params_, w=900, h=700)
if exists(_filename_): _igp_.loadLayout(_filename_)
_igp_

In [ ]:
_igp_.saveLayout(_filename_)